# Tradeoff Analysis

In [ ]:
import os
import re

import fl_const
import fl_util

fl_util.initialize()

JSON_POSTFIX = '.json'
COST_CSV_POSTFIX = 'cost.csv'

# def analyzeTradeoff(modelName, dataName, algName, nodeType, edgeType, tau1, tau2, numGroupsList):
#     fileNameBase = modelName + '_' + dataName + '_' + algName + '_' + nodeType + edgeType + '_' + \
#                     str(tau1) + '_' + str(tau2)
def analyzeTradeoff(scaleFactor, modelName, dataName, algName, nodeType, edgeType, tau1, tau2, numGroupsList):
    fileNameBase = 'scaleFactor=' + str(scaleFactor) + '_' + modelName + '_' + dataName + '_' + algName + '_' + nodeType + edgeType + '_' + \
                    str(tau1) + '_' + str(tau2)
    print(fileNameBase)
    print('#groups,cost,d_group,d_global,accuracy')
    for numGroups in numGroupsList:
        groupFileNameBase = fileNameBase + '_' + str(numGroups)
        jsonFilePath = os.path.join(fl_const.LOG_DIR_PATH, groupFileNameBase + JSON_POSTFIX)
                    
        # cost 파일을 읽고 최종 cost 추출
        fileCost = open(os.path.join(fl_const.LOG_DIR_PATH, groupFileNameBase + '_' + COST_CSV_POSTFIX), 'r')
        lines = fileCost.readlines()
        tokens = lines[-1].rstrip('\n').split(',')
        cost = float(tokens[3])
        
        # time 파일을 읽고 최종 accuracy 추출
        for fileName in os.listdir(fl_const.LOG_DIR_PATH):
            if fileName.startswith(groupFileNameBase) and fileName.endswith(fl_const.TIME_CSV_POSTFIX):
                fileTime = open(os.path.join(fl_const.LOG_DIR_PATH, fileName), 'r')
                break
        lines = fileTime.readlines()
        tokens = lines[-1].rstrip('\n').split(',')
        acc = float(tokens[2])
        
#         for fileName in os.listdir(fl_const.LOG_DIR_PATH):
#             if fileName.startswith(groupFileNameBase) and fileName.endswith(fl_const.TIME_CSV_POSTFIX):
#                 m = re.match(groupFileNameBase + '_(\d+)_(\d+)_0.(\d+)_' + fl_const.TIME_CSV_POSTFIX, fileName)
#                 if m: acc = int(m.group(3)) / 10
        
        # json 정보 추출
        json = fl_util.loadJson(jsonFilePath)
        print('%d,%.3f,%.3f,%.3f,%.3f' % (numGroups, cost, json['d_group'], json['d_global'], acc))
    print()
    
modelName = '2nn'
dataName = 'mnist-o'
algName = 'fedavg-ic'
tau1 = 1
tau2 = 5
for scaleFactor in [1.0, 2.0]:
    analyzeTradeoff(scaleFactor, modelName, dataName, algName, 't', 't', tau1, tau2, [2, 6, 10, 12])